Mount to google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


extraction of the meme dataset

In [2]:
import zipfile as zf
files = zf.ZipFile("/content/drive/MyDrive/FBHM.zip", 'r')
files.extractall('memes')
files.close()

importing libraries

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import LxmertTokenizer, LxmertModel
from PIL import Image
from torchvision import transforms
import json
import os
from tqdm import tqdm


CUDA is used for utilizing the GPU (for faster processing we have purchased colab pro)

imported torch

In [4]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.5.1+cu121
CUDA available: True


importing the pretrained LxMert Model (Learning Cross-Modality Encoder Representations from Transformers),LxMert tokenized

In [5]:
from transformers import LxmertModel, LxmertTokenizer
LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")
LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.3.bias', 'answer_head.logit_fc.3.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.decoder_dict.feat.bias', 'obj_predict_head.decoder_dict.feat.weight', 'obj_predict_head.decoder_dict.obj.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.transform.LayerNorm.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.bias', 'obj_pred

tokenizer_config.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

LxmertTokenizer(name_or_path='unc-nlp/lxmert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

checking it cuda is present and use GPU

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


a custom PyTorch dataset class ToxicMemeDataset for loading and processing a dataset of toxic memes, where each data sample includes text, an image, and a label

In [ ]:

class ToxicMemeDataset(Dataset):
    def __init__(self, json_file, img_dir, tokenizer, max_len, transform=None):
        with open(json_file, "r") as f:
            self.data = json.load(f)
        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]


        text = item["text"]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )


        img_path = os.path.join(self.img_dir, item["img"])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)


        label = torch.tensor(item["label"], dtype=torch.long)

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "image": image,
            "label": label,
        }


neural network model ToxicMemeClassifier that combines textual and visual inputs to classify toxic memes.

In [ ]:

class ToxicMemeClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ToxicMemeClassifier, self).__init__()
        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")

        # Add a ResNet backbone for visual feature extraction
        resnet = models.resnet50(pretrained=True)
        self.cnn_backbone = nn.Sequential(*list(resnet.children())[:-1])  # Remove final FC layer
        self.visual_fc = nn.Linear(resnet.fc.in_features, self.lxmert.config.visual_feat_dim)


        self.fc = nn.Linear(self.lxmert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, images):

        visual_feats = self.cnn_backbone(images).squeeze(-1).squeeze(-1)
        visual_feats = self.visual_fc(visual_feats).unsqueeze(1)


        batch_size = visual_feats.size(0)
        visual_pos = torch.zeros(batch_size, 1, 4).to(visual_feats.device)


        outputs = self.lxmert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            visual_feats=visual_feats,
            visual_pos=visual_pos,
        )
        logits = self.fc(outputs.pooled_output)
        return logits


training function train_model for training the ToxicMemeClassifier

In [ ]:

def train_model(model, data_loader, optimizer, criterion, device, epochs):
    model = model.to(device)
    for epoch in range(epochs):
        model.train()
        loop = tqdm(data_loader, desc=f"Epoch {epoch + 1}/{epochs}")
        total_loss = 0
        for batch in loop:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask, images)
            loss = criterion(outputs, labels)


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())
        print(f"Epoch {epoch + 1}: Loss = {total_loss / len(data_loader)}")


evaluation function, evaluate_model, to assess the performance of the ToxicMemeClassifier.

In [ ]:

def evaluate_model(model, data_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask, images)
            _, preds = torch.max(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Accuracy: {correct / total:.2f}")


combines the dataset, model, training, and evaluation components to execute the full pipeline for the toxic meme classification task.

In [8]:

if __name__ == "__main__":

    json_file = "/content/123 (1).json"
    img_dir = "/content/memes/FBHM/data/"


    max_len = 128
    batch_size = 16
    learning_rate = 1e-5
    epochs = 5


    tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])


    dataset = ToxicMemeDataset(json_file, img_dir, tokenizer, max_len, transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)


    model = ToxicMemeClassifier(num_classes=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


    train_model(model, train_loader, optimizer, criterion, device, epochs)
    evaluate_model(model, test_loader, device)


Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.3.bias', 'answer_head.logit_fc.3.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.decoder_dict.feat.bias', 'obj_predict_head.decoder_dict.feat.weight', 'obj_predict_head.decoder_dict.obj.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.transform.LayerNorm.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.bias', 'obj_pred

Epoch 1: Loss = 0.6382594274072086


Epoch 2/5: 100%|██████████| 425/425 [02:42<00:00,  2.62it/s, loss=0.568]


Epoch 2: Loss = 0.5610239753302406


Epoch 3/5: 100%|██████████| 425/425 [02:42<00:00,  2.62it/s, loss=0.253]


Epoch 3: Loss = 0.42629132645971635


Epoch 4/5: 100%|██████████| 425/425 [02:42<00:00,  2.62it/s, loss=0.391]


Epoch 4: Loss = 0.2547824035584927


Epoch 5/5: 100%|██████████| 425/425 [02:44<00:00,  2.58it/s, loss=0.0798]


Epoch 5: Loss = 0.15013612422653858
Accuracy: 0.67


The test_model function is designed to evaluate the performance of the ToxicMemeClassifier on a test dataset

In [9]:
def test_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["image"].to(device)
            labels = batch["label"].to(device)


            outputs = model(input_ids, attention_mask, images)
            _, preds = torch.max(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())


    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.2f}")

    return all_preds, all_labels


 calling the test_model function to evaluate the trained model on the test dataset.

In [10]:

test_preds, test_labels = test_model(model, test_loader, device)


Test Accuracy: 0.67


saves the trained model's parameters (weights)

In [11]:
torch.save(model.state_dict(), "toxic_meme_classifier.pth")

load the trained model and set it up for inference

In [12]:
model.load_state_dict(torch.load("toxic_meme_classifier.pth"))
model.eval()

<ipython-input-12-08da9a888559>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("toxic_meme_classifier.pth"))


ToxicMemeClassifier(
  (lxmert): LxmertModel(
    (embeddings): LxmertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LxmertEncoder(
      (visn_fc): LxmertVisualFeatureEncoder(
        (visn_fc): Linear(in_features=2048, out_features=768, bias=True)
        (visn_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (box_fc): Linear(in_features=4, out_features=768, bias=True)
        (box_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer): ModuleList(
        (0-8): 9 x LxmertLayer(
          (attention): LxmertSelfAttentionLayer(
            (self): LxmertAttention(
              (query):

predict function is designed to make predictions on a single meme (image and text) using the trained ToxicMemeClassifier

In [13]:
def predict(model, img_path, text, tokenizer, transform, device):
    model = model.to(device)

    encoding = tokenizer(
        text,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)


    image = Image.open(img_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)


    with torch.no_grad():
        output = model(input_ids, attention_mask, image)
        _, prediction = torch.max(output, dim=1)
        return prediction.item()


install easyocr

In [14]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.6/286.6 kB 24.8 MB/s eta 0:00:00


extracting text from an image and using the ToxicMemeClassifier to predict whether the meme is toxic or not.

In [18]:
import easyocr
from PIL import Image

def extract_text_from_image(img_path):

    reader = easyocr.Reader(['en'])


    results = reader.readtext(img_path)


    text = " ".join([result[1] for result in results])

    return text.strip()


img_path = "/content/48106.png"


text = "fuck islam"
print(f"\n Extracted Text: {text}")


prediction = predict(model, img_path, text, tokenizer, transform, device)
print(f"Prediction: {'Toxic' if prediction == 1 else 'Non-toxic'}")



 Extracted Text: fuck islam
Prediction: Toxic
